# Indicateur du vivre mieux par rapport au PIB
(Life satisfaction and GDP per capita)

## Objectif

Le but de cette première partie du travail est de créer une base de données SQL compilant les données par pays de l'indicateur du vivre mieux et du PIB.
Le PIB sera exprimé en € et l'indicateur en unité définies par l'OCDE.
Les noms des pays seront donnés en français.

Le but est de
- créer la base de donnée
- extraire et convertir les données souhaités
- alimenter la base de donnée

## Structure de la base de données MYSQL

1 table avec 3 colonnes: pays, PIB, indice

## Travail préliminaire sur les ressources

### PIB par habitant, valeurs issues des données du FMI
Dataset obtained from the IMF's website at: http://goo.gl/j1MSKe

In [1]:
import pandas as pd

In [2]:
gdp_per_capita = pd.read_csv("oecd.csv", 
                             thousands=',', delimiter='\t',
                             encoding='latin1', na_values="n/a")

FileNotFoundError: [Errno 2] File b'ressources/indicateur-du-vivre-mieux/gdp_per_capita.csv' does not exist: b'ressources/indicateur-du-vivre-mieux/gdp_per_capita.csv'

In [8]:
gdp_per_capita.rename(columns={"2015": "GDP", 'Country':'Country_Name_en'}, inplace=True)
#gdp_per_capita.head()

In [9]:
gdp_per_capita = gdp_per_capita[['Country_Name_en', 'GDP']]
gdp_per_capita.head()
#gdp_per_capita.dtypes

,Country_Name_en,GDP
0,Afghanistan,599.994
1,Albania,3995.383
2,Algeria,4318.135
3,Angola,4100.315
4,Antigua and Barbuda,14414.302


### Conversion en €

Taux de conversion retenu:
1 USD  = 0,92094 EUR au 31 décembre 2015

In [10]:
gdp_per_capita['GDP'] = gdp_per_capita['GDP'] * 0.92094
gdp_per_capita.head()

,Country_Name_en,GDP
0,Afghanistan,552.558474
1,Albania,3679.508020
2,Algeria,3976.743247
3,Angola,3776.144096
4,Antigua and Barbuda,13274.707284


### Indicateur du vivre mieux: better life satisfaction (BLI)
- This dataset was obtained from the OECD's website at: http://stats.oecd.org/index.aspx?DataSetCode=BLI
- Utilisation des filtres du site et de l'option En/Fr



In [14]:
life_sat = pd.read_csv("Ressources/indicateur-du-vivre-mieux/oecd_bli_2015_fr.csv", thousands=',')

In [15]:
life_sat = life_sat[['LOCATION', 'Value']]
life_sat.rename(columns={'LOCATION':'Country_Code', 'Value':'BLI'}, inplace=True)
life_sat.head()

,Country_Code,BLI
0,AUS,7.3
1,AUT,7.1
2,BEL,6.9
3,CAN,7.4
4,CZE,6.7


# Codes pays - Libellé français / anglais

Export depuis le site de l'OCDE des mêmes données en français puis en anglais

In [17]:
code_pays = pd.read_csv("Ressources/indicateur-du-vivre-mieux/oecd_bli_2015_fr.csv")

In [18]:
code_pays = code_pays[['LOCATION', 'Pays']]
code_pays.rename(columns={'LOCATION':'Country_Code', 'Pays':'Country_Name_fr'}, inplace=True)
code_pays.head()

,Country_Code,Country_Name_fr
0,AUS,Australie
1,AUT,Autriche
2,BEL,Belgique
3,CAN,Canada
4,CZE,République tchèque


In [20]:
country_code = pd.read_csv("Ressources/indicateur-du-vivre-mieux/oecd_bli_2015_en.csv")

In [21]:
country_code = country_code[['LOCATION', 'Country']]

In [22]:
country_code.rename(columns={'LOCATION':'Country_Code', 'Country':'Country_Name_en'}, inplace=True)
country_code.head()

,Country_Code,Country_Name_en
0,AUS,Australia
1,AUT,Austria
2,BEL,Belgium
3,CAN,Canada
4,CZE,Czech Republic


- Equivalent du inner join de sql: merge 
- [infos ici](https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html)
- Permet de faire une jointure sur deux dataframes (ici sur la colonne Country_ISO)

In [23]:
countries = pd.merge(left=code_pays, right=country_code, left_on='Country_Code', right_on='Country_Code')
countries.head()

,Country_Code,Country_Name_fr,Country_Name_en
0,AUS,Australie,Australia
1,AUT,Autriche,Austria
2,BEL,Belgique,Belgium
3,CAN,Canada,Canada
4,CZE,République tchèque,Czech Republic


## Aggrégation des données pour préparer les insertions dans la base

On fait une jointure sur le nom du pays en anglais (seule info que l'on a dans la table du FMI)

In [24]:
gdp_ISO = pd.merge(left=gdp_per_capita, right=countries, left_on='Country_Name_en', right_on='Country_Name_en')
gdp_ISO.head()

,Country_Name_en,GDP,Country_Code,Country_Name_fr
0,Australia,46932.819953,AUS,Australie
1,Austria,40267.209109,AUT,Autriche
2,Belgium,36935.801674,BEL,Belgique
3,Brazil,7984.547958,BRA,Brésil
4,Canada,39906.136163,CAN,Canada


In [26]:
# Vérification
gdp_per_capita.loc[gdp_per_capita['Country_Name_en'] == 'Australia']

,Country_Name_en,GDP
7,Australia,46932.819953


In [27]:
# Dataframe final pour la prise en compte du BLI
df = pd.merge(left=gdp_ISO, right=life_sat, left_on='Country_Code', right_on='Country_Code')
df.head()

,Country_Name_en,GDP,Country_Code,Country_Name_fr,BLI
0,Australia,46932.819953,AUS,Australie,7.3
1,Austria,40267.209109,AUT,Autriche,7.1
2,Belgium,36935.801674,BEL,Belgique,6.9
3,Brazil,7984.547958,BRA,Brésil,6.4
4,Canada,39906.136163,CAN,Canada,7.4


In [28]:
# Réduction aux seules colonnes utiles
df = df[['Country_Code', 'Country_Name_fr', 'GDP', 'BLI']]
df.head()

,Country_Code,Country_Name_fr,GDP,BLI
0,AUS,Australie,46932.819953,7.3
1,AUT,Autriche,40267.209109,7.1
2,BEL,Belgique,36935.801674,6.9
3,BRA,Brésil,7984.547958,6.4
4,CAN,Canada,39906.136163,7.4


In [29]:
# Un peu de nettoyage pour finir
# et on limite les valeurs du PIB à trois chiffres après la virgule
df.rename(columns={ 'Country_Name_fr':'Country_Name'}, inplace=True)
df = df.round({'GDP':2})

In [30]:
df.head()

,Country_Code,Country_Name,GDP,BLI
0,AUS,Australie,46932.82,7.3
1,AUT,Autriche,40267.21,7.1
2,BEL,Belgique,36935.80,6.9
3,BRA,Brésil,7984.55,6.4
4,CAN,Canada,39906.14,7.4


In [31]:
df.shape

(40, 4)

## Création de la base de données

```
DROP SCHEMA IF EXISTS ocde1;
CREATE SCHEMA ocde1;
USE ocde1;

CREATE TABLE ocde1.idvm (
    NP INT NOT NULL AUTO_INCREMENT,
    pays VARCHAR(45) NULL,
    PIB DECIMAL(10 , 2 ) NULL,
    indice DECIMAL(3 , 1 ) NULL,
    PRIMARY KEY (NP)
);
```

Utilisation d'une clé primaire numérique qui s'auto incrémente lors de chaque insertion d'un nouvel enregistrement dans la base (l'autre option serait d'utiliser directement le code ISO du pays que l'on sait unique par définition).

<img src="images/ocde1-db.png" style="width: 150px;"/>

### Insertion de données avec pymysql

[Documentation sur l'insertion de données depuis pandas dans MYSQL](https://www.dataquest.io/blog/sql-insert-tutorial/)

[Documentation de pymysql](https://pypi.org/project/PyMySQL/)

In [35]:
!pip install pymysql

In [36]:
import pymysql

In [37]:
# Autre option possible: sqlalchemy 

In [38]:
df_to_mysql = df[['Country_Name', 'GDP', 'BLI']]
df_to_mysql.head()

,Country_Name,GDP,BLI
0,Australie,46932.82,7.3
1,Autriche,40267.21,7.1
2,Belgique,36935.80,6.9
3,Brésil,7984.55,6.4
4,Canada,39906.14,7.4


In [39]:
import pymysql


# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='password',
                         db='ocde1')


# create cursor
cursor=connection.cursor()

In [40]:
# Insert DataFrame records one by one.
for i,row in df_to_mysql.iterrows():
    sql = "INSERT INTO `idvm` (`pays`,`PIB`,`indice`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [41]:
connection.close()

Vérification de la bonne insertion des données: `select * from ocde1.idvm;`